In [1]:
from importlib import reload
import honours_project.simulation as simulation
reload(simulation)
import honours_project.data as data
reload(data)

import json

In [223]:
from honours_project.data import PNE, Cycle, SimulationResult

# Example of problematic seed
sim = simulation.Simulation(5, 5)
result1 = sim.run()
result2 = sim.run()
print(json.dumps(result1, indent=4, cls=data.EnhancedJSONEncoder))
print(json.dumps(result2, indent=4, cls=data.EnhancedJSONEncoder))

pne_count = 0
cycle_count = 0
for _ in range(100):
    sim = simulation.Simulation(5, 5)
    result = sim.run()
    if isinstance(result, PNE):
        pne_count += 1
    elif isinstance(result, Cycle):
        cycle_count += 1

print(f"PNE count: {pne_count}")
print(f"Cycle count: {cycle_count}")

{
    "time": 0.02561916600097902,
    "iteration": 2
}
{
    "time": 0.007210000010672957,
    "iteration": 0
}
PNE count: 0
Cycle count: 0
